In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from nltk import word_tokenize
from functools import lru_cache
import re
import unicodedata
import sys
from collections import Counter
import nltk
nltk.download('brown')
from nltk.corpus import brown
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:

Dataset = namedtuple('Dataset', 'name, train, test')
Model = namedtuple('Model', 'type, name, dimension, corpus, model')

In [3]:
import urllib.request

url = "https://www.inf.uni-hamburg.de/en/inst/ab/lt/resources/data/complex-word-identification-dataset/cwishareddataset.zip"
filename = "cwishareddataset.zip"

urllib.request.urlretrieve(url, filename)
print("Download complete!")


Download complete!


In [4]:
import zipfile

zip_path = "cwishareddataset.zip"  # Ensure this is the correct filename

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(".")  # Extract to the current directory

print("Extraction complete!")


Extraction complete!


In [5]:
import os

MAIN_PATH_DATASET = "traindevset/english/"

# Check for all expected files
expected_files = ['Wikipedia_Train.tsv', 'Wikipedia_Dev.tsv',
                  'WikiNews_Train.tsv', 'WikiNews_Dev.tsv',
                  'News_Train.tsv', 'News_Dev.tsv']

for file in expected_files:
    file_path = os.path.join(MAIN_PATH_DATASET, file)
    print(f"Checking: {file_path} -> Exists? {os.path.exists(file_path)}")


Checking: traindevset/english/Wikipedia_Train.tsv -> Exists? True
Checking: traindevset/english/Wikipedia_Dev.tsv -> Exists? True
Checking: traindevset/english/WikiNews_Train.tsv -> Exists? True
Checking: traindevset/english/WikiNews_Dev.tsv -> Exists? True
Checking: traindevset/english/News_Train.tsv -> Exists? True
Checking: traindevset/english/News_Dev.tsv -> Exists? True


In [6]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 200)
MAIN_PATH_DATASET = "traindevset/english/"
genres = ['Wikipedia', 'WikiNews', 'News']
datasets = ['Train', 'Dev']
columns = ['id', 'sentence', "start", "end", "target", 
           "nat", "non_nat", "nat_marked", "non_nat_marked", "binary", "prob"]


datasets = [Dataset('Wikipedia', 'Train', 'Dev'),
            Dataset('WikiNews', 'Train', 'Dev'),
            Dataset('News', 'Train', 'Dev')]

feature_categories = []

def load_df(path):
    df = pd.read_csv(path, header=None, sep = "\t")
    df.columns = columns
    return df

datasets = [Dataset(d.name, load_df(MAIN_PATH_DATASET + d.name + '_' + d.train + '.tsv'),
                            load_df(MAIN_PATH_DATASET + d.name + '_' + d.test + '.tsv'))
                            for d in datasets]

In [ ]:
import requests

url = "http://nlp.stanford.edu/data/glove.6B.zip"
filename = "glove.6B.zip"

response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(filename, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Download complete!")
else:
    print("Server is down. Try again later.")


In [ ]:
import os

file_path = "glove.6B.zip"

if os.path.exists(file_path):
    os.remove(file_path)
    print("Corrupted file deleted. Download it again.")
else:
    print("No corrupted file found.")


In [ ]:
with zipfile.ZipFile("glove.6B.zip", "r") as zip_ref:
    zip_ref.extractall("glove/")

print("Extraction complete!")


In [ ]:
import os

print("Current Directory:", os.getcwd())
print("Files in Directory:", os.listdir())


In [ ]:
import shutil

source = "C:/Users\pc\Downloads\glove.6B.zip"  # Update this with the actual location
destination = os.getcwd()  # Moves to your current working directory

shutil.move(source, destination)
print("File moved successfully!")


In [ ]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

MAIN_PATH = 'embeddings/'


glove_models = []

glove_defs = [ Model('glove', 'glove.6B.300d.txt', 300, 'wikipedia+gigaword5', None)]
              
for model in glove_defs:
    glove_file = MAIN_PATH + model.name
    tmp_file = get_tmpfile(model.name + '-temp')
    glove2word2vec(glove_file, tmp_file)
    vecs = KeyedVectors.load_word2vec_format(tmp_file)
    glove_models.append(Model(model.type, model.name, model.dimension, model.corpus, vecs))
    print('load model : {}'.format(model.name))
    
print(glove_models)

In [ ]:
import os

file_path = "embeddings/glove.6B.300d.txt"

if os.path.exists(file_path):
    print("File found:", file_path)
else:
    print("File not found! Check your directory.")


In [ ]:
import os

def find_glove_file(root_folder):
    for dirpath, _, filenames in os.walk(root_folder):
        if "glove.6B.300d.txt" in filenames:
            return os.path.join(dirpath, "glove.6B.300d.txt")
    return None

file_path = find_glove_file("C:/")  # Change to "/" for Mac/Linux

if file_path:
    print("Found GloVe file at:", file_path)
else:
    print("File not found. Please download it again.")


In [ ]:
import zipfile

zip_path = "glove.6B.zip"

if os.path.exists(zip_path):
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall("embeddings/")
    print("Extraction complete!")
else:
    print("ZIP file not found. Download it first.")


In [ ]:
import os

file_path = "embeddings/glove.6B.300d.txt"

if os.path.exists(file_path):
    print("✅ File is ready to use!")
else:
    print("❌ File is still missing. Check again.")


In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

glove_file = "embeddings/glove.6B.300d.txt"
tmp_file = "embeddings/glove.6B.300d.word2vec"

glove2word2vec(glove_file, tmp_file)
vecs = KeyedVectors.load_word2vec_format(tmp_file)

print("Model loaded successfully!")


In [ ]:
dataframe = datasets[0].train[0:30]

def overlaps(start1, end1, start2, end2):
    return bool(range(max(start1, start2), min(end1, end2)+1))

def extract_ngrams_group(group):
    targets = zip(group['target'].values.tolist(), group['start'].values.tolist(),
                 group['end'].values.tolist(), group['binary'].values.tolist())
    for word, start, end, binary in targets:
        tokens = word.split()
        if len(tokens)>1:
            olap_words = [(w, b) for w, s, e, b in targets if overlaps(start, end, s, e)]
            
    
grouped = dataframe.groupby('sentence').apply(lambda group : extract_ngrams_group(group))

In [ ]:
wordlist_lowercased = set(i.lower() for i in brown.words())
print (len(wordlist_lowercased))

In [ ]:
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P'))

def remove_punctuation(text):
    return text.translate(tbl)

@lru_cache(maxsize=None)
def all_tokens_with_index(context):
    curr_pos = 0
    targets = []
    j = 0
    w = 0
    curr_split = ''
    ctx_split = context.split()
    whitespaces = re.findall('\s+', context)
    num_whitespaces = [len(token) for token in whitespaces]
    num_whitespaces.append(1)
    tokens = word_tokenize(context)
    tokens = ['"' if token not in context else token for token in tokens]
    for index, token in enumerate(tokens, 1):
        targets.append((token, index, curr_pos, (curr_pos + len(token))))
        curr_pos += len(token)
        curr_split += token
        if ctx_split[j] == curr_split:
            curr_pos += num_whitespaces[w]
            j += 1
            w += 1
            curr_split = ''
    return [val for val in targets if val[0] != '"']

def build_vocabulary(sentences, embedding_model, dimension):
    all_words = [tpl[0] for sentence in sentences for tpl in sentence['seq']] + list(wordlist_lowercased)
    print('# Words : {}'.format(len(all_words)))
    counter = Counter(all_words)
    vocab_size = len(counter) + 1
    print('# Vocab : {}'.format(vocab_size))
    print('# embeding model  : {}'.format(len(embedding_model.vocab)))   
    word2index = {word : index for index, (word, count) in enumerate(counter.most_common(), 1)}
    index2word = {index : word for word, index in word2index.items()}
    # +1 required for pad token
    embedding_matrix = np.zeros(((vocab_size), dimension))
    missing_embed_words = []
    i_ = 0
    for word, index in word2index.items():
        if word in embedding_model.vocab:
            embedding = embedding_model[word]
        else:
             i_ +=1
             continue
        embedding_matrix[index] = embedding
    missing_embed_count = len(missing_embed_words)
    print('# Words missing embedding : {}'.format(missing_embed_count))
    print('Embedding shape : {}'.format(embedding_matrix.shape))
    print("i: ", i_ )
    return word2index, index2word, embedding_matrix

def forward_transformation(dataframe, lowercase = True, filter_punc = True, filtering = "a132"):
    grouped = dataframe.groupby('sentence').apply(lambda row : 
                        {'sent_id' : list(set(row['sent_id']))[0],
                         'sentence' : list(set(row['sentence']))[0], 
                         'tags': [tag for tag in zip(row['target'], 
                            row['start'], row['end'], row['binary'], row['prob'])]})
    sentences = []
    for vals in grouped:
        sent_id = vals['sent_id']
        sentence = vals['sentence']
        tags = vals['tags']
        tags_without_labels = [(word, start, end) for word, start, end, binary, prob in tags]
        all_tokens = all_tokens_with_index(sentence)
        sent_repr = [(word, start, end, tags[tags_without_labels.index((word, start, end))][3],
                     tags[tags_without_labels.index((word, start, end))][4])
           if (word, start, end) in tags_without_labels 
          else (word, start, end, 0, 0.0) for word, index, start, end in all_tokens]
        if lowercase:
            sent_repr = [(word.lower(), start, end, binary, prob) 
                         for word, start, end, binary, prob in sent_repr]
        if filter_punc:
            sent_repr = list(filter(lambda vals : remove_punctuation(vals[0]), sent_repr))
        if filtering:
            sent_repr = list(filter(lambda vals : vals[0] != "'s", sent_repr))
            sent_repr = list(filter(lambda vals : vals[0] != "``", sent_repr))
        sentences.append({'sent_id' : sent_id, 'sentence' : sentence, 'seq' : sent_repr})
    return sentences

def split_sentence_seqs(sentences):
    words, start_end, binary, prob = [], [], [] ,[]
    for sent in sentences:
        sequence = sent['seq']
        curr_w, curr_se, curr_b, curr_p = map(list, zip(*[(vals[0], 
            (vals[1], vals[2]), vals[3], vals[4]) for vals in sequence]))
        words.append(curr_w)
        start_end.append(curr_se)
        binary.append(curr_b)
        prob.append(curr_p)
    return words, start_end, binary, prob

In [ ]:
datasets.append(Dataset('train_all_test_wiki', 
        datasets[0].train.append(datasets[1].train).append(datasets[2].train), datasets[0].test))

In [ ]:
# Append train and test set
dataset_sel = datasets[3]
train_num_rows = dataset_sel.train.shape[0]
train_num_sents = len(list(set(dataset_sel.train.sentence.values.tolist())))

test_num_rows = dataset_sel.test.shape[0]
test_num_sents = len(list(set(dataset_sel.test.sentence.values.tolist())))

dataset = dataset_sel.train.append(dataset_sel.test)
dataset['sent_id'] = dataset.groupby('sentence').ngroup()
dataset_num_rows = dataset.shape[0]
dataset_num_sents = len(list(set(dataset.sentence.values.tolist())))

print('# Rows train : {}'.format(train_num_rows))
print('# Rows test : {}'.format(test_num_rows))
print('# Rows dataset : {}'.format(dataset_num_rows))

print('# Sents train : {}'.format(train_num_sents))
print('# Sents test : {}'.format(test_num_sents))
print('# Sents dataset : {}'.format(dataset_num_sents))

In [ ]:
dataset

In [ ]:
sentences = forward_transformation(dataset)
train_sentences = sentences[:train_num_sents]
test_sentences = sentences[train_num_sents:]

In [ ]:
words, start_end, binary, prob = split_sentence_seqs(sentences)

In [ ]:
sentence_lens = [len(sent) for sent in words]

In [ ]:
def build_vocabulary(sentences, embedding_model, dimension):
    from collections import Counter
    import numpy as np

    counter = Counter()
    total_words = 0

    for item in sentences:
        if isinstance(item, dict):
            if 'sentence' in item:
                words = item['sentence'].lower().split()  # Tokenize sentence
            elif 'seq' in item:
                words = [word_info[0] for word_info in item['seq']]  # Extract words from 'seq'
            else:
                raise ValueError(f"Unexpected dictionary format: {item}")

            counter.update(words)
            total_words += len(words)  # Track total words
        else:
            raise TypeError(f"Expected dictionary, but got {type(item)}: {item}")

    vocab_size = len(counter) + 1  # +1 for padding/indexing consistency
    print("# Words : {}".format(total_words))
    print("# Vocab : {}".format(vocab_size))
    print("# embeding model  : {}".format(len(embedding_model.key_to_index)))

    word2index = {word: index for index, (word, count) in enumerate(counter.most_common(), 1)}
    index2word = {index: word for word, index in word2index.items()}

    embedding = np.zeros((vocab_size, dimension))
    missing_words = 0

    for word, index in word2index.items():
        if word in embedding_model.key_to_index:
            embedding[index] = embedding_model[word]
        else:
            missing_words += 1

    print("# Words missing embedding : {}".format(missing_words))
    print("Embedding shape : {}".format(embedding.shape))
    print("i: ", word2index.get('i', 'Not found'))  # Print the index of "i" if it exists

    return word2index, index2word, embedding


In [ ]:
word2index, index2word, embedding = build_vocabulary(sentences, embedding_model, dimension)


In [ ]:
missing_words = 0

for word, index in word2index.items():
    cleaned_word = word.lower().strip()  # Normalize word
    if cleaned_word in embedding_model.key_to_index:
        embedding[index] = embedding_model[cleaned_word]
    else:
        missing_words += 1


In [ ]:
word = word.lower()


In [ ]:
import re
word = re.sub(r'[^a-zA-Z]', '', word)  # Keep only alphabets


In [ ]:
missing_words = [word for word in word2index.keys() if word not in embedding_model.key_to_index]
print("Sample missing words:", missing_words[:20])
print("Total missing words:", len(missing_words))


In [ ]:
missing_words = 0
for word, index in word2index.items():
    cleaned_word = word.lower().strip()  # Ensure consistency
    if cleaned_word in embedding_model.key_to_index:
        embedding[index] = embedding_model[cleaned_word]
    else:
        missing_words += 1

print("Final missing words:", missing_words)


In [ ]:
import re

def clean_word(word):
    return re.sub(r'[^a-zA-Z]', '', word).lower()  # Remove numbers/punctuation

missing_words = 0
for word, index in word2index.items():
    cleaned_word = clean_word(word)
    if cleaned_word in embedding_model.key_to_index:
        embedding[index] = embedding_model[cleaned_word]
    else:
        missing_words += 1

print("Missing words after cleaning:", missing_words)


In [ ]:
import re
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from collections import Counter

# Function to normalize words
def normalize_word(word):
    word = str(word).lower()  # Convert to lowercase
    word = re.sub(r"[^a-zA-Z0-9]", "", word)  # Remove special characters
    return word

# Build vocabulary
def build_vocabulary(sentences, embedding_model, dimension):
    counter = Counter()

    for sentence in sentences:
        if isinstance(sentence, dict) and 'seq' in sentence:  # Ensure sentence is a valid dict with 'seq' key
            words = [normalize_word(word[0]) for word in sentence['seq']]  # Extract and normalize words
            counter.update(words)
        else:
            raise TypeError(f"Expected dict with 'seq' key, but got {type(sentence)}: {sentence}")

    vocab_size = len(counter) + 1
    print('# Vocab : {}'.format(vocab_size))
    print('# embedding model  : {}'.format(len(embedding_model.key_to_index)))  

    # Create word-to-index mappings
    word2index = {word: index for index, (word, count) in enumerate(counter.most_common(), 1)}
    index2word = {index: word for word, index in word2index.items()}

    # Initialize embedding matrix
    embedding = np.zeros((vocab_size, dimension))
    missing_words = 0

    for word, index in word2index.items():
        if word in embedding_model.key_to_index:
            embedding[index] = embedding_model[word]
        else:
            missing_words += 1  # Track missing words

    print(f'# Words missing embedding : {missing_words}')
    print(f'Embedding shape : {embedding.shape}')

    return word2index, index2word, embedding

# Normalize and map words to indices
def convert_words_to_indices(words, word2index):
    words_with_indices = []
    
    for sent in words:
        indexed_sentence = [word2index.get(normalize_word(word), 0) for word in sent]  # Map to index (default 0 if missing)
        words_with_indices.append(indexed_sentence)
    
    return words_with_indices

# Main Execution
embedding_model = glove_models[0].model
dimension = embedding_model.vector_size

word2index, index2word, embedding = build_vocabulary(sentences, embedding_model, dimension)

# Normalize and convert words to indices
words_with_indices = convert_words_to_indices(words, word2index)

# Get sentence lengths
sent_lens = [len(sentence['seq']) for sentence in sentences]
sent_max_length = np.max(sent_lens)
print('Max length sentence : {}'.format(sent_max_length))

# Padding sequences
words_padded = pad_sequences(maxlen=sent_max_length, sequences=words_with_indices, padding="post", value=0)
binary_padded = pad_sequences(maxlen=sent_max_length, sequences=binary, padding="post", value=0)
prob_padded = pad_sequences(maxlen=sent_max_length, sequences=prob, padding="post", value=0, dtype="float")

# Convert binary labels to categorical format
binary_padded_categorical = [to_categorical(clazz, num_classes=2) for clazz in binary_padded]

# Debugging: Print some normalized words and indices
print("Sample normalized words:", [[normalize_word(word) for word in sent] for sent in words[:5]])
print("Sample indexed words:", words_with_indices[:5])


In [ ]:

# (1) Training set
train_words_padded = words_padded[:train_num_sents]
train_binary_padded = binary_padded[:train_num_sents]
train_binary_padded_categorical = binary_padded_categorical[:train_num_sents]
train_prob_padded = prob_padded[:train_num_sents]
train_start_end = start_end[:train_num_sents]

# (2) Test set
test_words_padded = words_padded[train_num_sents:]
test_binary_padded = binary_padded[train_num_sents:]
test_binary_padded_categorical = binary_padded_categorical[train_num_sents:]
test_prob_padded = prob_padded[train_num_sents:]
test_start_end = start_end[train_num_sents:]

print('Training set length : {}'.format(len(train_words_padded)))
print('Test set length : {}'.format(len(test_words_padded)))

In [ ]:
from sklearn.metrics import f1_score
import keras.callbacks
from keras import backend as K
class Metrics(keras.callbacks.Callback):
    def __init__(self, validation_data):
        self.f1_scores = []
        self.validation_data = validation_data
        
    def on_epoch_end(self, batch, logs={}):
        predict = np.asarray(self.model.predict(self.validation_data[0]))
        targ = self.validation_data[1]
        targ = np.array(targ)
        shape = targ.shape
        targ = targ.reshape((shape[0]*shape[1], shape[2]))
        targ = np.argmax(targ, axis = 1)
        predict = predict.reshape((shape[0]*shape[1]), shape[2])
        predict = np.argmax(predict, axis = 1)
        self.f1s=f1_score(targ, predict)
        print("\nF1 Score:")
        print(f1_score(targ, np.ones(shape[0]*shape[1])))
        self.f1_scores.append(self.f1s)
        return

In [ ]:
from keras.models import Model
from keras.layers import Input

from keras import backend as K
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

vocab_size = embedding.shape[0]
dimension = embedding.shape[1]

np.set_printoptions(threshold=np.inf)

in_seq = Input(shape=(sent_max_length,))
embed = Embedding(input_dim=vocab_size, output_dim=dimension, \
                  weights=[embedding], input_length=sent_max_length)(in_seq)
drop = Dropout(0.1)(embed)
lstm = Bidirectional(LSTM(units=150, return_sequences=True, recurrent_dropout=0.1))(drop)
out = TimeDistributed(Dense(2, activation="softmax"))(lstm) 

model = Model(in_seq, out)

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["accuracy"])
model.summary()

metrics = Metrics((test_words_padded, np.array(test_binary_padded_categorical)))


In [ ]:
history = model.fit(train_words_padded, np.array(train_binary_padded_categorical), batch_size=10, 
                    epochs=3, validation_data = (test_words_padded, np.array(test_binary_padded_categorical)), 
                    verbose=1, callbacks=[metrics])

In [ ]:
import os

cwd = os.getcwd()
print("Saving to:", cwd)

model_save_name = 'model_CWI_full.h5'
path_dir = os.path.join(cwd, model_save_name)

from keras.models import load_model

model.save(path_dir)


In [ ]:
model_save_name = 'model_CWI_full.h5'
path_dir = ""

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")
epoch_f1s = plt.plot(metrics.f1_scores)
plt.show()

In [ ]:
stop_words_ = set(stopwords.words('english'))
def cleaner(word):
  #Remove links
  word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', 
                '', word, flags=re.MULTILINE)
  word = re.sub('[\W]', ' ', word)
  word = re.sub('[^a-zA-Z]', ' ', word)
  return word.lower().strip()

In [ ]:

def process_input(input_text):
  input_text = cleaner(input_text)
  clean_text = []
  index_list =[]
  input_token = []
  index_list_zipf = []
  for i, word in enumerate(input_text.split()):
    if word in word2index:
      clean_text.append(word)
      input_token.append(word2index[word])
    else:
      index_list.append(i)
  input_padded = pad_sequences(maxlen=sent_max_length, sequences=[input_token], padding="post", value=0)
  return input_padded, index_list, len(clean_text)


In [ ]:
def complete_missing_word(pred_binary, index_list, len_list):
  list_cwi_predictions = list(pred_binary[0][:len_list])
  for i in index_list:
    list_cwi_predictions.insert(i, 0)
  return list_cwi_predictions


In [ ]:
!pip install transformers
!pip install torch

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

bert_model = 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model)
model = BertForMaskedLM.from_pretrained(bert_model)
model.eval()

In [ ]:
!pip install wordfreq

In [ ]:
from wordfreq import zipf_frequency
zipf_frequency('stop', 'en')


In [ ]:
from wordfreq import zipf_frequency
zipf_frequency('thwart', 'en')

In [ ]:


def get_bert_candidates(input_text, list_cwi_predictions, numb_predictions_displayed = 10):
  list_candidates_bert = []
  for word,pred  in zip(input_text.split(), list_cwi_predictions):
    if (pred and (pos_tag([word])[0][1] in ['NNS', 'NN', 'VBP', 'RB', 'VBG','VBD' ]))  or (zipf_frequency(word, 'en')) <3.1:
      replace_word_mask = input_text.replace(word, '[MASK]')
      text = f'[CLS]{replace_word_mask} [SEP] {input_text} [SEP] '
      tokenized_text = tokenizer.tokenize(text)
      masked_index = [i for i, x in enumerate(tokenized_text) if x == '[MASK]'][0]
      indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
      segments_ids = [0]*len(tokenized_text)
      tokens_tensor = torch.tensor([indexed_tokens])
      segments_tensors = torch.tensor([segments_ids])
      # Predict all tokens
      with torch.no_grad():
          outputs = model(tokens_tensor, token_type_ids=segments_tensors)
          predictions = outputs[0][0][masked_index]
      predicted_ids = torch.argsort(predictions, descending=True)[:numb_predictions_displayed]
      predicted_tokens = tokenizer.convert_ids_to_tokens(list(predicted_ids))
      list_candidates_bert.append((word, predicted_tokens))
  return list_candidates_bert
  



In [ ]:
list_texts = [ 
 'The Risk That Students Could Arrive at School With the Coronavirus As schools grapple with how to reopen, new estimates show that large parts of the country would probably see infected students if classrooms opened now.',
 'How a photograph of a young man cradling his dying friend sent me on a journey across India.',
 'Pro-democracy parties, which had hoped to ride widespread discontent to big gains, saw the yearlong delay as an attempt to thwart them.',
 'Night after night, calm gave way to chaos. See what happened between the protesters and the federal agents.',
 'Contact Tracing Is Failing in Many States. Here is Why. Inadequate testing and protracted delays in producing results have crippled tracking and hampered efforts to contain major outbreaks.',
 'After an initial decrease in the youth detention population, the rate of release has slowed, and the gap between white youth and Black youth has grown.'
 'A laboratory experiment hints at some of the ways the virus might elude antibody treatments. Combining therapies could help, experts said.',
 'Though I may not be here with you, I urge you to answer the highest calling of your heart and stand up for what you truly believe.',
 'The research does not prove that infected children are contagious, but it should influence the debate about reopening schools, some experts said.',
 'Dropping antibody counts are not a sign that our immune system is failing against the coronavirus, nor an omen that we can not develop a viable vaccine.',
 'The Senate majority leader has said he will not approve a stimulus package without a “liability shield,” but top White House officials say they do not see it as essential.',
 'Campaign efforts to refocus come as the president continues to push divisive messages that have frustrated his own party.'
] 

In [ ]:
for input_text in list_texts:
  new_text = input_text
  input_padded, index_list, len_list = process_input(input_text)
  pred_cwi = model.predict(input_padded)
  pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
  complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
  bert_candidates =   get_bert_candidates(input_text, complete_cwi_predictions)
  for word_to_replace, l_candidates in bert_candidates:
    tuples_word_zipf = []
    for w in l_candidates:
      if w.isalpha():
        tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
    tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
    new_text = re.sub(word_to_replace, tuples_word_zipf[0][0], new_text) 
  print("Original text: ", input_text )
  print("Simplified text:", new_text, "\n")